## Creating confidence intervals in python
In this assessment, you will look at data from a study on toddler sleep habits. 

The confidence intervals you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 

Reference: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
# Import the data (use this if running your Jupyter notebook within Coursera)
df = pd.read_csv("nap_no_nap.csv") 

In [0]:
# Imort the data (uncomment the line below and use this if you downloaded the Jupyter notebook)
# df = pd.read_csv("https://raw.githubusercontent.com/UMstatspy/UMStatsPy/master/Course_2/nap_no_nap.csv")

In [4]:
# First, look at the DataFrame to get a sense of the data
df.head()

,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
0,1,female,33.7,19.24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.45,20.68,0.23,1.92,7.17,629.4,643.0,629.4,-1.21,-1.44,6.68,11.93
1,2,female,31.5,18.27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.23,19.48,0.25,1.09,6.69,672.4,700.4,672.4,-0.96,-1.21,6.82,12.42
2,3,male,31.9,19.14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.60,20.05,0.45,1.29,6.53,628.8,682.6,628.8,-0.46,-0.91,6.15,11.39
3,4,female,31.6,19.69,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.46,19.50,0.05,1.89,8.28,766.6,784.0,766.6,0.23,0.19,6.20,12.59
4,5,female,33.0,19.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.21,19.65,0.45,1.30,6.95,678.0,718.0,678.0,0.31,-0.13,5.78,11.43


**Question**: What variable is used in the column 'napping' to indicate a toddler takes a nap?  
**Question**: What is the sample size $n$? What is the sample size for toddlers who nap, $n_1$, and toddlers who don't nap, $n_2$?

In [7]:
df["napping"]

0     0
1     0
2     0
3     0
4     0
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
Name: napping, dtype: int64

In [13]:
df[df["napping"] == 1].reset_index(drop=True)["napping"].value_counts()

1    15
Name: napping, dtype: int64

### Average bedtime confidence interval for napping and non napping toddlers
Create two 95% confidence intervals for the average bedtime, one for toddler who nap and one for toddlers who don't.

Before any analysis, we will convert 'night bedtime' into decimalized time.

In [14]:
# Convert 'night bedtime' into decimalized time
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [22]:
bedtime_nap = df[df["napping"] == 1]["night bedtime"].reset_index(drop=True)
bedtime_nap.head()

0    1235.0
1    1260.0
2    1321.0
3    1224.0
4    1278.0
Name: night bedtime, dtype: float64

In [24]:
bedtime_no_nap = df[df["napping"] == 0]["night bedtime"].reset_index(drop=True)
bedtime_no_nap

0    1245.0
1    1163.0
2    1200.0
3    1186.0
4    1161.0
Name: night bedtime, dtype: float64

Now find the sample mean bedtime for nap and no_nap.

In [25]:
nap_mean_bedtime = bedtime_nap.mean()
nap_mean_bedtime

1233.0666666666666

In [26]:
no_nap_mean_bedtime = bedtime_no_nap.mean()
no_nap_mean_bedtime

1191.0

Now find the standard error for $\bar{X}_{nap}$ and $\bar{X}_{no\ nap}$.

In [29]:
n_nap = 15
std_nap = bedtime_nap.std()
nap_se_mean_bedtime = std_nap / np.sqrt(n_nap)
nap_se_mean_bedtime

8.893800230479801

In [30]:
n_no_nap = 5
std_no_nap = bedtime_no_nap.std()
no_nap_se_mean_bedtime = std_no_nap / np.sqrt(n_no_nap)
no_nap_se_mean_bedtime

15.339491516996253

**Question**: Given our sample sizes of $n_1$ and $n_2$ for napping and non napping toddlers respectively, how many degrees of freedom ($df$) are there for the associated $t$ distributions?

To build a 95% confidence interval, what is the value of t\*?  You can find this value using the percent point function: 
```
from scipy.stats import t

t.ppf(probabiliy, df)
```
This will return the quantile value such that to the left of this value, the tail probabiliy is equal to the input probabiliy (for the specified degrees of freedom). 

Example: to find the $t^*$ for a 90% confidence interval, we want $t^*$ such that 90% of the density of the $t$ distribution lies between $-t^*$ and $t^*$.

Or in other words if $X \sim t(df)$:

P($-t^*$ < X < $t^*$) = .90

Which, because the $t$ distribution is symmetric, is equivalent to finding $t^*$ such that:  

P(X < $t^*$) = .95

So the $t^*$ for a 90% confidence interval, and lets say df=10, will be:

t_star = t.ppf(.95, df=10)


In [31]:
from scipy.stats import t

In [57]:
# Find the t_stars for the 95% confidence intervals
nap_t_star = t.ppf(.95, df=14)

In [58]:
no_nap_t_star = t.ppf(.95, df=4)

In [69]:
nap_t_star, no_nap_t_star = 2.144, 2.776

**Quesion**: What is $t^*$ for nap and no nap?

Now to create our confidence intervals. For the average bedtime for nap and no nap, find the upper and lower bounds for the respective confidence intervals.

In [70]:
f =  nap_mean_bedtime - nap_t_star * nap_se_mean_bedtime
s = nap_mean_bedtime + nap_t_star * nap_se_mean_bedtime
f,s

(1213.998358972518, 1252.1349743608152)

In [72]:
f_nn =  no_nap_mean_bedtime - no_nap_t_star * no_nap_se_mean_bedtime
s_nn = no_nap_mean_bedtime + no_nap_t_star * no_nap_se_mean_bedtime
f_nn,s_nn

(1148.4175715488184, 1233.5824284511816)

**Question**: What are the 95% confidence intervals, rounded to the nearest ten, for the average bedtime (in decimalized time) for toddlers who nap and for toddlers who don't nap? 

CI = $\bar{X} \pm \ t^* \cdot s.e.(\bar{X})$

**Challenge problem**: Write a function that inputs the column containing the data you want to build your confidence interval from and returns the confidence interval as a list or tuple (i.e. \[upper, lowe\] or (upper, lower)).